In [1]:
from langchain_community.llms import Ollama
import pandas as pd
import langchain_core
from langchain.globals import set_verbose, set_debug
import matplotlib.pyplot as plt
import re
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import os
from langchain_core.prompts import PromptTemplate
import seaborn as sns
import inspect
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import traceback
import numpy as np
from sklearn.model_selection import train_test_split 
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI, ChatOpenAI
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from treelib import Node, Tree  #  moved here to prevent bugs

import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AutoKaggle Project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [2]:
os.environ["OPENAI_API_KEY"] = 'sk-proj-XXXX'
os.environ["OPENAI_ORGANIZATION"] = 'org-XXXX'

### Validations

In [3]:
class MaxRetriesExceededError(Exception):
    pass
class MissingCodeValidationException(Exception):
    pass

class FailedGeneratingFunction(Exception):
    pass
class MissingFunctionBoundaries(Exception):
    pass

In [4]:
class PandasDataFrameValidationException(Exception):
    pass

class PandasColumnsIntegerValidationException(Exception):
    pass
class TargetFeatureInDataFrameValidationException(Exception):
    pass
class TargetFeatureIsBinaryValidationException(Exception):
    pass



In [5]:
class PredictInModelObjectValidationException(Exception):
    pass
class CheckFittedValidationException(Exception):
    pass
class FunctionDoesNotMatchSignatureValidationException(Exception):
    pass
class ModelIsPipelineValidationException(Exception):
    pass
def check_model_is_not_pipeline(model):
    try:
        assert type(model) != sklearn.pipeline.Pipeline
    except:
        raise ModelIsPipelineValidationException("Returned model should not be a pipeline")
def predict_in_model_object(model):
    try:
        assert 'predict' in dir(model)
    except:
        raise PredictInModelObjectValidationException("Returned object is not a classifier")
def check_fitted(model): 
    try:
        assert hasattr(model, "classes_")
    except:
        raise CheckFittedValidationException("Returned object was not fitted.")

In [6]:
def prediction_prevent_babbling(llm, prompt):
    chunks = []
    min_characters = 100
    char_count = 0
    print(f"------PROMPT-------:\n {prompt} \n -------------")
    for chunk in llm.stream(prompt):
        if type(chunk) != str:
            chunk = chunk.content
        char_count += len(chunk)
        #print(chunk, end="", flush=True)
        # We stop the model when it begings babbling, usually marked by the <|end_header_id|> token.
        if '<|end_header_id|>' in chunk and char_count > min_characters:
            break
        chunks.append(chunk)
    return "".join(chunks)

In [7]:
parsing_regex = {"llama3":"```(?:[Pp]ython)?\n([^`]+?)```",
                "openai": "(def.*)"
                }
OUTPUT_VARIABLE_PROMPT = ".\nCall your function and save it to a variable named {}"

In [8]:

class AutoKaggleCoT:
    def __init__(self, llm, llm_type, dataset_name, dataset, dataset_info, target_feature, metric,
                 max_self_debug_attempts=3, max_retry_attempts=10): #llm_type
        self.init_template = f"""Hello, today your job is to be a data scientist assistant for classic ML problems. 
                            You will provide working functions that can be executed with no required changes.
                            Try to be creative in your code.
                            dataset info:
                            {dataset_info}
                            
                            New Task:
                            """
        self.llm = llm
        self.llm_type = llm_type
        self.pdf = dataset
        self.metric = metric
        self.max_self_debug_attempts = max_self_debug_attempts
        self.max_retry_attempts = max_retry_attempts
        self.target_feature = target_feature
        results_dict[llm_type] = {} 

    def pandas_dataframe_validation(self, pdf):
        try:
            assert type(pdf) == pd.DataFrame
        except:
            raise PandasDataFrameValidationException("Function output is not a pandas dataframe")
    def pandas_columns_integers_validation(self, pdf):
        try:
            pdf.astype(int)
        except:
            raise PandasColumnsIntegerValidationException("Not all the dataframe's columns are numerical")
    def target_feature_in_dataframe(self, pdf):
        try:
            assert self.target_feature in pdf.columns.tolist()
        except:
            raise TargetFeatureInDataFrameValidationException(f"Target feature '{self.target_feature}' is not in the dataframe")
    def target_feature_is_binary(self, pdf):
        try:
            assert pdf[self.target_feature].tolist() == original_value
        except:
            raise TargetFeatureIsBinaryValidationException(f"Target feature '{self.target_feature}' was altered")

    def read_prompt(self, prompt_name):
        with open(f'./prompts/flow_prompts/{prompt_name}.txt', 'r') as prompt_file:
            return prompt_file.read()
        
    def predict_prompt_and_parse(self, prompt):
        
        res = prediction_prevent_babbling(self.llm, prompt)
        if '```' in res:
            regex = parsing_regex['llama3']
        else:
            regex = parsing_regex['openai']
        code_blocks = re.findall(regex, res, flags=re.DOTALL)
        return code_blocks

        
    def execute_code(self, code_blocks, output_variable_name, **input_variables):
        ldic = locals()
        for variable_name, variable_value in input_variables.items():
            globals()[variable_name] = variable_value
        for code in code_blocks:
            print("-------------Executing", code)
            exec(code, globals())    
        
        return eval(output_variable_name)

    def manage_error(self, error_count, retry_attempts, code_blocks, e, step_prompt):
        error_count += 1
        if error_count >= self.max_self_debug_attempts:
            print("Failed self debug",)
            prompt = step_prompt
            retry_attempts += 1
            error_count = 0
            print("Retry attempt: ", retry_attempts)
            if retry_attempts > self.max_retry_attempts:
                raise MaxRetriesExceededError()
        else:
            prompt = f"History: {step_prompt}, you gave me {code_blocks}. I got the error {type(e)}:{e}, can you fix it?"
        return prompt, error_count, retry_attempts

    def validate_step(self,step_output, step_validations):
        for validation in step_validations:
            validation(step_output)
            
    def run_step(self, step_prompt, step_name, output_variable_name, step_validations = None, **input_variables):
               
        completed = False
        error_count = 0
        retry_attempts = 0
        prompt = step_prompt
        results_dict[self.llm_type][step_name] = {'generation_error_count': 0, 'debug_error_count': 0}
        while not completed:
            code_blocks = self.predict_prompt_and_parse(self.init_template + prompt+ OUTPUT_VARIABLE_PROMPT.format(output_variable_name))  
            
            try:
                if len(code_blocks) == 0:
                    raise MissingCodeValidationException()
                step_output = self.execute_code(code_blocks, output_variable_name, **input_variables)
                if step_validations:
                    self.validate_step(step_output, step_validations)
            except Exception as e:
                results_dict[self.llm_type][step_name][type(e).__name__] = results_dict[self.llm_type][step_name].get(type(e).__name__,0) + 1
                print("Error:", e, '---')
                
                if error_count == 0:
                    results_dict[self.llm_type][step_name]['generation_error_count'] += 1
                else:
                    results_dict[self.llm_type][step_name]['debug_error_count'] += 1
                prompt, error_count, retry_attempts = self.manage_error(error_count, retry_attempts, code_blocks, e, step_prompt)
                    
            else:
                completed = True
        return step_output

    def llm_exploration(self, pdf):
        self.exploration_string = self.run_step(self.read_prompt('exploration'), 'exploration', 'exploration_string')
        print("Finished exploration", self.exploration_string)
        return self.exploration_string

    def llm_preprocessing(self, exploration_string, pdf, target_feature):
        self.preprocessed_data = self.run_step(self.read_prompt('data_preprocessing').format(exploration_string)
                                               + '\nreturn a pandas dataframe', 'preprocessing', 'preprocessed_data',
                                               step_validations=[self.pandas_dataframe_validation, self.pandas_columns_integers_validation,
                                                                self.target_feature_in_dataframe, self.target_feature_is_binary] ,pdf = pdf)
        print("Finished preprocessing")
        X = self.preprocessed_data.drop(columns=[target_feature])
        y = self.preprocessed_data[target_feature]
        return X,y

    def llm_training(self, X_train, y_train):
        self.best_model = self.run_step(self.read_prompt('model_training'), 'training', 'best_model',
                                        step_validations=[predict_in_model_object],
                                        X_train=X_train, y_train=y_train)
        print("Finished training", self.best_model)
        return self.best_model

    def run(self):
        exploration_string = self.llm_exploration(self.pdf)
        X, y = self.llm_preprocessing(exploration_string, self.pdf, self.target_feature)       
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X,y,test_size=0.25, random_state=42)
        best_model = self.llm_training(self.X_train, self.y_train)
        print(f"{self.evaluate(best_model, self.X_test, self.y_test)} accuracy score")
        results_dict[self.llm_type]['best_score'] = self.evaluate(best_model, self.X_test, self.y_test)
    
        
        
    def evaluate(self, llm_model, X_test, y_test):
        if self.metric == 'classification':
            y_pred = llm_model.predict(X_test)
            model_score = accuracy_score(y_test, y_pred)
            return model_score
        else:
            y_pred = llm_model.predict(X_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            return rmse

In [9]:
got_openai_api_access = False  # change this if true.
got_codegemma = True
got_llama3 = False
llms = []
if got_openai_api_access:
    llms += [('gpt-4o', ChatOpenAI(model='gpt-4o')),
           ('gpt-4-turbo', ChatOpenAI(model='gpt-4-turbo')), ('gpt-4', ChatOpenAI(model='gpt-4')),
           ('gpt-3.5-turbo', ChatOpenAI(model='gpt-3.5-turbo'))]
if got_codegemma:
    llms += [('codegemma', Ollama(model='codegemma'))]
if got_llama3:
    llms += [('llama3', Ollama(model='llama3'))]

In [10]:
def get_dataset(dataset_name):
    prompt_path = './prompts/flow_prompts/'
    if dataset_name == 'housing':
        target_feature = 'SalePrice'
        metric = 'regression'
        pdf = pd.read_csv('./data/house-prices-advanced-regression-techniques/train.csv')  
        with open(os.path.join(prompt_path, 'houses_prompt.txt')) as prompt_f:
            dataset_info = prompt_f.read()
    elif dataset_name == 'titanic':
        target_feature = 'Survived'
        metric = 'classification'
        pdf = pd.read_csv('./data/titanic/train.csv')
        with open(os.path.join(prompt_path, 'titanic_prompt.txt')) as prompt_f:
            dataset_info = prompt_f.read()
    return (metric, target_feature, pdf, dataset_info)

In [ ]:
results_dict = {} 



for dataset_name in ['titanic', 'housing']:
    for llm_type, llm in llms:
        pdf = None
        metric, target_feature, pdf, dataset_info = get_dataset(dataset_name)
        original_value = pdf[target_feature].tolist()
        pdf_temp = pdf.copy()
        AK = AutoKaggleCoT(llm, llm_type, dataset_name, pdf_temp, dataset_info, target_feature = target_feature, metric=metric, max_self_debug_attempts=3, max_retry_attempts=12)
        AK.run()

------PROMPT-------:
 Hello, today your job is to be a data scientist assistant for classic ML problems. 
                            You will provide working functions that can be executed with no required changes.
                            Try to be creative in your code.
                            dataset info:
                            This dataset title is:
Titanic - Machine Learning from Disaster
Size: 93.08 kB
Data Dictionary
Variable	Definition	Key         Feature Type
Survived	Survived	0 = No, 1 = Yes         Number
Pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd       Number
Sex	Sex                                             String
Name    Name of the passenger, may also contain a title String
Age	Age in years                                    Number
Sibsp	# of siblings / spouses aboard the Titanic      Number
Parch	# of parents / children aboard the Titanic      Number
Ticket	Ticket number                                   Number
Fare	Passenger fare                     

<string>:12: SyntaxWarning: invalid escape sequence '\.'
<string>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


<string>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)

-------------Executing import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def preprocess_titanic_data(pdf):
    # Handle missing data
    pdf['Age'].fillna(pdf['Age'].mean(), inplace=True)
    pdf['Cabin'].fillna('N/A', inplace=True)

    # Feature engineering using exploration string
    pdf['FamilySize'] = pdf['SibSp'] + pdf['Parch'] + 1
    pdf['Title'] = pdf['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    pdf['IsAlone'] = (pdf['FamilySize'] == 1)

    # Encode categorical variables
    cat_cols = ['Sex', 'Embarked', 'Title']
    num_cols = pdf.columns.difference(cat_cols)

    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(), cat_cols),
        ('num', StandardScaler(), num_cols)
    ])

    preprocessed_data = preprocessor.fit_transform(pdf)

    return preprocessed_data

# Assuming pdf is already loaded as a pandas dataframe
preprocessed_data = preprocess_titanic_data(pdf)



<string>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




-------------Executing import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def preprocess_titanic_data(pdf):
    # Handle missing data
    pdf['Age'].fillna(pdf['Age'].mean(), inplace=True)
    pdf['Cabin'].fillna('N/A', inplace=True)

    # Feature engineering using exploration string
    pdf['FamilySize'] = pdf['SibSp'] + pdf['Parch'] + 1
    pdf['Title'] = pdf['Name'].str.extract(' ([A-Za-z]+)\\\\.', expand=False)
    pdf['IsAlone'] = (pdf['FamilySize'] == 1)

    # Encode categorical variables
    cat_cols = ['Sex', 'Embarked', 'Title']
    num_cols = pdf.columns.difference(cat_cols)

    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(), cat_cols),
        ('num', StandardScaler(), num_cols)
    ])

    preprocessed_data = preprocessor.fit_transform(pdf)

    return preprocessed_data

# Assuming pdf is already loaded as a pandas dataframe
preprocessed_data = preprocess_titanic_data(pdf)

...

-------------Executing import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

def preprocess_data(pdf):
    # Handle missing data
    pdf['Age'].fillna(pdf['Age'].mean(), inplace=True)
    pdf['Cabin'].fillna('Unknown', inplace=True)

    # Feature engineering
    pdf['FamilySize'] = pdf['SibSp'] + pdf['Parch'] + 1
    pdf['Title'] = pdf['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

    # Encode categorical variables
    le = LabelEncoder()
    pdf['Sex'] = le.fit_transform(pdf['Sex'])
    pdf['Embarked'] = le.fit_transform(pdf['Embarked'])
    pdf['Title'] = le.fit_transform(pdf['Title'])

    # Scale numerical features
    scaler = StandardScaler()
    pdf['Age'] = scaler.fit_transform(pdf['Age'].values.reshape(-1, 1))
    pdf['Fare'] = scaler.fit_transform(pdf['Fare'].values.reshape(-1, 1))

    # Remove categorical variables
    pdf.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

    return pdf

-------------Executing preprocessed_data =

<string>:11: SyntaxWarning: invalid escape sequence '\.'
<string>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


<string>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)

-------------Executing from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

# Define the models
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'SVC': SVC()
}

# Define the hyperparameter grid
param_grid = {
    'RandomForestClassifier': [{'n_estimators': [100, 500, 1000], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10] }],
    'AdaBoostClassifier': [{'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5] }],
    'SVC': [{'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1]}]
}

def find_best_model(X_train, y_train):
    best_model = None
    best_score = float('-inf')

    # Iterate through the models
    for model_name, model in models.items():
        # Define the gridsearch object
        gridsearch = GridSearchCV(estimator=model, param_grid=param_grid[model_name], cv=5, n_jobs=-1

C:\Users\Inbar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Finished training RandomForestClassifier(max_depth=20, min_samples_split=5)
0.8430493273542601 accuracy score
------PROMPT-------:
 Hello, today your job is to be a data scientist assistant for classic ML problems. 
                            You will provide working functions that can be executed with no required changes.
                            Try to be creative in your code.
                            dataset info:
                            This dataset title is:
House Prices - Advanced Regression Techniques
Size: 957.39 kB
Data fields
SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
MSSubClass: The building class
MSZoning: The general zoning classification
LotFrontage: Linear feet of street connected to property
LotArea: Lot size in square feet
Street: Type of road access
Alley: Type of alley access
LotShape: General shape of property
LandContour: Flatness of the property
Utilities: Type of utilities available
Lot

# ToT

In [ ]:
BFS = 0
DFS = 1
class AutoKaggleToT:
    def __init__(self, llm, llm_type, dataset_name, dataset, dataset_info, target_feature, metric, max_self_debug_attempts=3,
                 max_retry_attempts=3, child_count=3, tree_scan_method=BFS): #llm_type
        self.llm = llm
        self.pdf = dataset
        self.metric = metric
        self.max_self_debug_attempts = max_self_debug_attempts
        self.max_retry_attempts = max_retry_attempts
        self.target_feature = target_feature
        self.child_count = child_count
        self.dataset_name = dataset_name
        if dataset_name not in results_tree:
            results_tree.create_node(dataset_name, dataset_name, parent='root')
        self.base_parent_name = f"{self.dataset_name}: {llm_type}: {self.max_self_debug_attempts}, {max_retry_attempts}, {child_count}"            
        results_tree.create_node(self.base_parent_name, self.base_parent_name, parent=dataset_name)
        self.init_template = f"""Hello, today your job is to be a data scientist assistant for classic ML problems. 
                                You will provide functions that can be executed with no required changes.
                                We will be working in a tree of thought manner, In each step I want you to suggest {child_count} different functions.
                                The functions should be named function_1, ..., function_{self.child_count}
                                Be creative in your solutions the functions need to be complex.
                                Each function should be indpendent and a full solution to the task given.
                                We will then try to execute them and debug each function separately.
                                dataset info:
                                {dataset_info}
                                
                                New Task:
                                """
                
        self.metric = metric

    def read_prompt(self, prompt_name):
        with open(f'./prompts/flow_prompts/tot_prompts/{prompt_name}.txt', 'r') as prompt_file:
            return prompt_file.read()
        
    def predict_prompt_and_parse(self, prompt):
        
        res = prediction_prevent_babbling(self.llm, prompt)
        if '```' in res:
            regex = parsing_regex['llama3']
        else:
            regex = parsing_regex['openai']
        code_blocks = re.findall(regex, res, flags=re.DOTALL)
        return code_blocks

    def execute_code(self, code_blocks, **input_variables):
        for variable_name, variable_value in input_variables.items():
            globals()[variable_name] = variable_value
        for code in code_blocks:
            print("-------------Executing", code)
            exec(code, globals())    

    def pandas_dataframe_validation(self, pdf):
        try:
            assert type(pdf) == pd.DataFrame
        except:
            raise PandasDataFrameValidationException("Function output is not a pandas dataframe")
    def pandas_columns_integers_validation(self, pdf):
        try:
            pdf.astype(int)
        except:
            raise PandasColumnsIntegerValidationException("Not all the dataframe's columns are numerical")
    def target_feature_in_dataframe(self, pdf):
        try:
            assert self.target_feature in pdf.columns.tolist()
        except:
            raise TargetFeatureInDataFrameValidationException(f"Target feature '{self.target_feature}' is not in the dataframe")
    def target_feature_is_binary(self, pdf):
        try:
            assert pdf[self.target_feature].tolist() == original_value
        except:
            raise TargetFeatureIsBinaryValidationException(f"Target feature '{self.target_feature}' was altered")

    def correct_function_signature(self, function, input_variables):
        try:
            assert list(function.__code__.co_varnames)[:function.__code__.co_argcount] == list(input_variables.keys())
        except:
            raise FunctionDoesNotMatchSignatureValidationException(f"Function does not match signature {list(function.__code__.co_varnames)[:function.__code__.co_argcount]} != {list(input_variables.keys())}")
    def manage_error(self, retry_prompt, initial_prompt, error_count, retry_attempts, retry_limit=-1, self_debug_attempts=-1):
        if retry_limit == -1:
            retry_limit=self.max_retry_attempts
        if self_debug_attempts == -1:
            self_debug_attempts = self.max_self_debug_attempts
        error_count += 1
        if error_count >= self_debug_attempts:
            print("Failed self debug")
            prompt = initial_prompt
            retry_attempts += 1
            error_count = 0
            print("Retry attempt: ", retry_attempts)
            if retry_attempts >= retry_limit:
                raise MaxRetriesExceededError()
        else:
            prompt = retry_prompt
        return prompt, error_count, retry_attempts

    def validate_step(self,step_output, step_validations):
        for validation in step_validations:
            print('---VALIDATION---')
            validation(step_output)
            
    def run_step(self, step_prompt, step_name, parent_node, step_validations = None, **input_variables):
        completed = False
        error_count = self.max_self_debug_attempts
        retry_attempts = 0
        prompt = self.init_template + step_prompt+ f"\nDO NOT CALL THE FUNCTIONS. \n Each function should complete the task independently and covers all bullet points \n function names should be in the format of function_1, ..., function_{self.child_count}"
        self.prompt = prompt
        step_outputs = []
        succeeded = True
        if type(results_tree[parent_node].data) != dict:
            results_tree[parent_node].data = {}
        while not completed:
            code_blocks = self.predict_prompt_and_parse(prompt)  
            
            try:
                print(f"-----Given code block------:\n {code_blocks} \n ------------")
                
                if len(code_blocks) == 0:
                    raise MissingCodeValidationException()
                self.execute_code(code_blocks, **input_variables)
                if len(code_blocks) == 1:
                    code_blocks = re.findall(r'(def function_\d.*?)(?:\n\n|```|$)', code_blocks[0], flags=re.DOTALL)
                
                for i in range(1,self.child_count+1):
                    function = eval(f'function_{i}')
                    self.correct_function_signature(function, input_variables)
                    
                for i in range(1,self.child_count+1):
                    step_outputs.append(self.execute_function(eval(f"function_{i}"), code_blocks[i-1], parent_node, i, step_name, step_validations, **input_variables))
                    
                if self.child_count / 2 < len([output for output in step_outputs if type(output) == type(None)]):
                    raise FailedGeneratingFunction()
            except Exception as e:
                try:
                    results_tree[parent_node].data['child_generation_error:' + type(e).__name__] = results_tree[parent_node].data.get(type(e).__name__,0) + 1
                    print("Error:", e, "------")
                    retry_prompt = ""
                    initial_prompt = prompt
                    prompt, error_count, retry_attempts = self.manage_error(retry_prompt, initial_prompt, error_count, retry_attempts, self_debug_attempts = 1)
                except Exception as e:
                    succeeded = False
                    completed = True
                    
            else:
                completed = True
        for i in range(1, self.child_count+1):
            node_name = f"{parent_node}: {step_name}_{i}"
            if node_name not in results_tree:
                results_tree.create_node(f"{step_name}_{i}", node_name, parent=parent_node, data={"succeeded":False, 'succeeded_on_self_debug': False})
        results_tree[parent_node].data['child_retry_attempts'] = retry_attempts        
        results_tree[parent_node].data['child_generation_succeeded'] = succeeded
        
        return step_outputs       
        
        
    def execute_function(self, function, function_code, parent_node, child_num, step_name, step_validations, **input_variables):
        
        function_name = function.__name__
        completed = False
        retry_attempts = 0
        error_count = 0
        code_error_count = 0
        should_execute_code = False
        succeeded = True
        attempted_self_debug = False
        node_data = {}
        while not completed:
            try:
                if should_execute_code:
                    function_code = "\n".join(code_blocks)
                    self.execute_code(code_blocks, **input_variables)
                    function = eval(function_name)
                    
                step_output = function(**input_variables)
                if step_validations:
                    self.validate_step(step_output, step_validations)
            except Exception as e:
                node_data[type(e).__name__] = node_data.get(type(e).__name__,0) + 1
                node_data['number_of_self_debug'] = node_data.get('number_of_self_debug', 0) + 1
                print("Error", e, '----')
                retry_prompt = f"{self.prompt} + \n You provided the code:\n ```Python\n{function_code}```. I received the error {type(e)}:{e}, can you regenerate? Call the function in the same name. \nDO NOT CALL THE FUNCTIONS. \n The function names should be in the format of function_1, ..., function_{self.child_count}"
                initial_prompt = f"{self.prompt} + \n You provided the code:\n ```Python\n{function_code}```. I have problems with it, can you generate different code that does the same thing? call the function in the same name. \n DO NOT CALL THE FUNCTIONS. \n function names should be in the format of function_1, ..., function_{self.child_count}"
                attempted_self_debug = True
                try:
                    prompt, error_count, retry_attempts = self.manage_error(retry_prompt, initial_prompt, error_count, retry_attempts, retry_limit=2)
                    code_blocks = self.predict_prompt_and_parse(prompt)  
                    should_execute_code = True
                except Exception as e:
                    step_output = None
                    succeeded = False
                    completed = True
                
            else:
                completed = True
        node_data['succeeded_on_self_debug'] =  succeeded & attempted_self_debug
        node_data['succeeded'] = succeeded
        node_name = f"{parent_node}: {step_name}_{child_num}"
        if node_name not in results_tree:
            results_tree.create_node(f"{step_name}_{child_num}", node_name, parent=parent_node, data=node_data)
        else:
            for key in list(node_data.keys()):
                if key in list(results_tree[node_name].data.keys()):
                    results_tree[node_name].data[key] += node_data[key]
                else:
                    results_tree[node_name].data[key] = node_data[key]
                    
            
        return step_output
        
    
    def llm_exploration(self, pdf, parent_name):
        self.exploration_string = self.run_step(self.read_prompt('exploration'), "exploration", parent_name, pdf=pdf)
        exploration_strings = [string[:4096] for string in self.exploration_string if string != None]
        print("Finished exploration", exploration_strings)
        return exploration_strings

    def llm_preprocessing(self, exploration_string, pdf, target_feature, parent_name):
        self.preprocessed_data_list = self.run_step(self.read_prompt('data_preprocessing').format(exploration_string)
                                               + '\nreturn a pandas dataframe', "preprocessing", parent_name,
                                               step_validations=[self.pandas_dataframe_validation, self.pandas_columns_integers_validation,
                                                                self.target_feature_in_dataframe, self.target_feature_is_binary] ,pdf = pdf)
        print("Finished preprocessing")
        Xs= []
        ys = []
        for preprocessed_data in self.preprocessed_data_list:
            if type(preprocessed_data) != type(None):
                if type(preprocessed_data) == pd.DataFrame and not preprocessed_data.empty:
                    Xs.append(preprocessed_data.drop(columns=[target_feature]))
                    ys.append(preprocessed_data[target_feature])
        return Xs,ys

    def llm_training(self, X_train, y_train, parent_name):
        self.best_model = self.run_step(self.read_prompt('model_training'), "training", parent_name,
                                        step_validations=[predict_in_model_object],
                                        X_train=X_train, y_train=y_train)
        print("Finished training", self.best_model)
        return [model for model in self.best_model if type(model) != type(None)]

    def run(self):
        exploration_strings = self.llm_exploration(self.pdf, parent_name=self.base_parent_name)
        best_model = None
        best_score = 0
        for i, exploration_string in enumerate(exploration_strings):
            exploration_parent_name = f"{self.base_parent_name}: exploration_{i+1}"
            Xs, ys = self.llm_preprocessing(exploration_string, self.pdf, self.target_feature, parent_name=exploration_parent_name)       
            for j, (X, y) in enumerate(zip(Xs, ys)):
                preprocessing_parent_name = f"{exploration_parent_name}: preprocessing_{j+1}"
                self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X,y,test_size=0.25, random_state=42)
                models = self.llm_training(self.X_train, self.y_train, parent_name=preprocessing_parent_name)
                for k, current_model in enumerate(models):
                    current_score = self.evaluate(current_model, self.X_test, self.y_test)
                    results_tree[f"{preprocessing_parent_name}: training_{k+1}"].data['score'] = current_score
                    print(f"{current_score} accuracy score")
                    if self.metric == 'classification':
                        if current_score > best_score:
                            best_score = current_score
                            best_model = current_model
                    else:
                        if current_score < best_score:
                            best_score = current_score
                            best_model = current_model
        results_tree[self.base_parent_name].data['best_score'] = best_score
        results_tree[self.base_parent_name].data['best_model'] = best_model
        print(f"Best score:{best_score}, best_model:{best_model}")
        return best_model
            
        
    def evaluate(self, llm_model, X_test, y_test):
        if self.metric == 'classification':
            y_pred = llm_model.predict(X_test)
            model_score = accuracy_score(y_test, y_pred)
            return model_score
        else:
            y_pred = llm_model.predict(X_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            return rmse

In [ ]:
results_tree = Tree()
results_tree.create_node('root', 'root')
    for dataset_name in ['housing','titanic']:
    for llm_type, llm in llms:
        metric, target_feature, pdf, dataset_info = get_dataset(dataset_name)
        original_value = pdf[target_feature].tolist()
        pdf_temp = pdf.copy()
        AK = AutoKaggleToT(llm, llm_type, dataset_name, pdf_temp, dataset_info=dataset_info, metric=metric,
        target_feature = target_feature, max_self_debug_attempts=2, max_retry_attempts=5, child_count=2)
        AK.run()

In [ ]:
results_tree.nodes